In [5]:
from importance_measures.bdds import Buddy, BuddyFunc, u_influence, u_blame, omega
import time

def time_op(func):
    def wrapper(*args, **kwargs):
        dt = time.time()
        val = func(*args, **kwargs)
        print(f"[{(time.time() - dt)*1000:010.4f} ms] {func.__name__}{args}")
        return val
    return wrapper

timed_u_influence = time_op(u_influence)
timed_u_blame = time_op(u_blame)
timed_omega = time_op(omega)

with Buddy(["x", "y", "z"]) as model:
    f = model.parse("x ^ (y & z)")
    infl = timed_u_influence(f, "y") * 2**-model.varcount
    blame = timed_u_blame(f, "z") * 2**-model.varcount
    g = timed_omega(f)
    g.dump("g.pdf")
    print(f"influence {infl:0.5f}, blame {blame:.5f}")

[00000.0350 ms] u_influence(<importance_measures.bdds.buddy.BuddyFunc object at 0x7f728d156140>, 'y')
[00000.6876 ms] u_blame(<importance_measures.bdds.buddy.BuddyFunc object at 0x7f728d156140>, 'z')
[00000.0720 ms] omega(<importance_measures.bdds.buddy.BuddyFunc object at 0x7f728d156140>,)
influence 0.50000, blame 0.70833
